In [17]:
import json
import csv
import pandas as pd

In [18]:
mappedDogBreeds = '''{
  "American Pit Bull Terrier/Pit Bull": "American Staffordshire Terrier",
  "Labrador Retriever Crossbreed": "Labrador Retriever",
  "Poodle, Standard": "Poodle",
  "German Shepherd Crossbreed": "German Shepherd",
  "German Shepherd Dog": "German Shepherd",
  "Schnauzer, Miniature": "Miniature Schnauzer",
  "Bull Dog, French": "French Bulldog",
  "Poodle, Toy": "Poodle",
  "American Pit Bull Mix / Pit Bull Mix": "American Staffordshire Terrier",
  "Poodle, Miniature": "Poodle",
  "Bull Dog, English": "Bulldog",
  "Dachshund Smooth Coat": "Dachshund",
  "Puggle": "Pug",
  "West High White Terrier": "West Highland White Terrier",
  "Beagle Crossbreed": "Beagle",
  "Labradoodle": "Labrador Retriever",
  "Wheaton Terrier": "Soft-coated Wheaten Terrier",
  "Silky Terrier": "Australian Silky Terrier",
  "Dachshund Smooth Coat Miniature": "Dachshund",
  "Collie, Border": "Border Collie",
  "American Eskimo dog": null,
  "Brussels Griffon": "Griffon Bruxellois",
  "Bassett Hound": "Basset Hound",
  "Schnauzer, Standard": "Standard Schnauzer",
  "Collie Crossbreed": "Collie",
  "Welsh Corgi, Pembroke": "Pembroke Welsh Corgi",
  "Cane Corso": null,
  "Cotton De Tulear": null,
  "Dachshund, Long Haired Miniature": "Dachshund",
  "Shar-Pei, Chinese": "Chinese Shar Pei",
  "Dachshund, Long Haired": "Dachshund",
  "Australian Cattledog": "Australian Cattle Dog",
  "Mastiff, Bull": "Bullmastiff",
  "Japanese Chin/Spaniel": "Japanese Chin",
  "Pointer, German Shorthaired": "German Shorthaired Pointer",
  "Brittany Spaniel": "Brittany",
  "Schipperkee": "Schipperke",
  "Akita Crossbreed": "Akita",
  "Jindo Dog, Korea": null,
  "Welsh Corgi, Cardigan": "Cardigan Welsh Corgi",
  "Dachshund, Wirehaired": "Dachshund",
  "Coonhound, Black and Tan": "Black and Tan Coonhound",
  "Dachshund, Wirehaired, Miniature": "Dachshund",
  "Collie, Bearded": "Bearded Collie",
  "Mastiff, Old English": "Mastiff",
  "Collie, Rough Coat": "Collie",
  "Mastiff, French (Dogue de Bordeaux)": "Mastiff",
  "Schnauzer, Giant": "Giant Schnauzer",
  "Collie, Smooth Coat": "Collie",
  "Coonhound, Treeing Walker": "Black and Tan Coonhound",
  "Mastiff, Neapolitan": "Mastiff",
  "Coonhound, Blue Tick": "Black and Tan Coonhound",
  "Belgian Griffon": "Wirehaired Pointing Griffon",
  "Pointer, German Wirehaired": "German Wirehaired Pointer",
  "Pharoh hound": "Pharaoh Hound",
  "Australian Kelpie": null,
  "Mastiff, Tibetan": "Mastiff",
  "Fila Brasileiro": null,
  "Russian Wolfhound": "Irish Wolfhound"
}'''

In [19]:
# convert mappedDogBreeds to pandas dataframe
df_mappedDogBreeds = pd.DataFrame(json.loads(mappedDogBreeds).items(), columns=['new', 'old'])

df_mappedDogBreeds.head()

,new,old
0,American Pit Bull Terrier/Pit Bull,American Staffordshire Terrier
1,Labrador Retriever Crossbreed,Labrador Retriever
2,"Poodle, Standard",Poodle
3,German Shepherd Crossbreed,German Shepherd
4,German Shepherd Dog,German Shepherd


In [20]:
#read from data/dog_intelligence.csv
dogIntelligence = pd.read_csv('data/dog_intelligence.csv')

dogIntelligence.head()

,breed,classification,obey,reps_lower,reps_upper
0,Border Collie,Brightest Dogs,0.95,1,4
1,Poodle,Brightest Dogs,0.95,1,4
2,German Shepherd,Brightest Dogs,0.95,1,4
3,Golden Retriever,Brightest Dogs,0.95,1,4
4,Doberman Pinscher,Brightest Dogs,0.95,1,4


In [21]:
#use replace dogIntelligence.breed with df_mappedDogBreeds.new
dogIntelligence['breed'] = dogIntelligence['breed'].replace(df_mappedDogBreeds.set_index('old')['new'])


In [22]:
dogIntelligence.head()

,breed,classification,obey,reps_lower,reps_upper
0,"Collie, Border",Brightest Dogs,0.95,1,4
1,"Poodle, Miniature",Brightest Dogs,0.95,1,4
2,German Shepherd Dog,Brightest Dogs,0.95,1,4
3,Golden Retriever,Brightest Dogs,0.95,1,4
4,Doberman Pinscher,Brightest Dogs,0.95,1,4


In [36]:
#save to csv    
dogIntelligence.to_csv('data/dog_intelligence_new.csv', index=False)

In [37]:
#remove quotes from csv
with open('data/dog_intelligence_new.csv', 'r') as infile, open('data/dog_intelligence_new2.csv', 'w') as outfile:
    data = infile.read()
    data = data.replace('"', '')
    outfile.write(data)